In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import warnings #supress warnings
warnings.filterwarnings(action="ignore")

# Any results you write to the current directory are saved as output.



![# Titanic](https://i.imgur.com/44aKU98.jpg)



KERNEL Making In progress

# 1. Introduction  




The aim of the competition is to correctly predict the survival rate of Titanic Survivors. However, I wanted this to be a good introduction to all beginners. So I clubbed most of the ML algorithms.  

But first I had to decide whether this task was going to be a done by a supervised ML Algorithm or  unsupervised ML algorithm. In all fairness even experts in the field are divided on this question so I used a mixture of both.

Next I needed to figure out if the supervised problem is a regression task or a classification task.; or incase of an unsupervised problem is it a clustering task or an association task.

Now that I have decided what kind of problems I need to boil this down to, let’s look at our dataset.




In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
#from math import pi
from collections import Counter
import numpy as np
import seaborn as sns

import warnings
warnings.filterwarnings('ignore') 

df = pd.read_csv('../input/train.csv')#extract values
Test = pd.read_csv('../input/test.csv',index_col = None)#do not drop passenger id
Y = df['Survived'] # dependent variable



![#Embedding and Exploring](https://i.imgur.com/e2qvzy0.jpg)




# 2. Exploratory Data Analysis

***Exploratory Data Analysis***

Exploratory Data Analysis (EDA) is an approach/philosophy for data analysis that employs a variety of techniques (mostly graphical) to
1. Maximize insight into a data set
2. Uncover underlying structure
3. Extract important variables
4. Detect outliers and anomalies
5. Test underlying assumptions
6. Develop parsimonious models
7. Determine optimal factor settings.

EDA is not identical to statistical graphics although the two terms are used almost interchangeably. Statistical graphics is a collection of techniques--all graphically based and all focusing on one data characterization aspect. EDA encompasses a larger venue; EDA is an approach to data analysis that postpones the usual assumptions about what kind of model the data follow with the more direct approach of allowing the data itself to reveal its underlying structure and model. EDA is not a mere collection of techniques; EDA is a philosophy as to how we dissect a data set; what we look for; how we look; and how we interpret. It is true that EDA heavily uses the collection of techniques that we call "statistical graphics", but it is not identical to statistical graphics per se.

### 2a. Determining the missing data
Missing data can occur because of nonresponse: no information is provided for one or more items or for a whole unit. So first we need to find the missing data and fill or replace the missing data with value. For this as of now, I use the median to fill the values or the most frequent to fill NA values.

**Some good references:**

https://en.wikipedia.org/wiki/Missing_data

https://machinelearningmastery.com/handle-missing-data-python/

https://www.programcreek.com/python/example/93356/sklearn.preprocessing.Imputer

In [ ]:
cols_with_missing = [col for col in df.columns if df[col].isnull().any()]
print(cols_with_missing)
cols_with_missing2 = [col for col in Test.columns if Test[col].isnull().any()]
print(cols_with_missing2)

In [ ]:
for col in cols_with_missing:
    if(df[col].dtype == np.dtype('O')):
         df[col]= df[col].fillna(df[col].value_counts().index[0]) 
   #replace nan with most frequent
    else:
        df[col] = df[col].fillna(0) 
        #replace nan with median
print(df.isnull().any())
print(Test.isnull().any())

In [ ]:

for col in cols_with_missing2:
    if(Test[col].dtype == np.dtype('O')):
         Test[col]= Test[col].fillna(Test[col].value_counts().index[0]) #replace nan with most frequent
    else:
        Test[col] = Test[col].fillna(0) 
        #replace nan with median
print(df.isnull().any())
print(Test.isnull().any())



### 2b. Encoding and Categorizing Data

Once we have gathered data, we might wish to classify it.  Roughly speaking, data can be classified as categorical data or quantitative data.

Categorical (qualitative) data are pieces of information that allow us to classify the objects under investigation into various categories. In this case the dataframe called 'Title' has been recategorized to 4 values *(Mr,Mrs,Miss,Master)*.

Quantitative data are responses that are numerical in nature and with which we can perform meaningful arithmetic calculations.



In [ ]:
#encoding and categorizng data

df['Title'] = df['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
Test['Title'] = Test['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
def replace_titles(x):
    title = x['Title']
    if title in ['Capt', 'Col', 'Don', 'Jonkheer', 'Major', 'Rev', 'Sir']:
        return 'Mr'
    elif title in ['the Countess', 'Mme', 'Lady']:
        return 'Mrs'
    elif title in ['Mlle', 'Ms']:
        return 'Miss'
    elif title =='Dr':
        if x['Sex']=='male':
            return 'Mr'
        else:
            return 'Mrs'
    else:
        return title

df['Title'] = df.apply(replace_titles,axis = 1)
Test['Title'] = Test.apply(replace_titles,axis = 1)
features= [ 'Title', 'Pclass','Sex','Fare','Embarked']
X = df[features]

### 2c. Data Visualization

Data visualization is viewed by many disciplines as a modern equivalent of visual communication. It involves the creation and study of the visual representation of data.

I use two highly effective libraries here

a. Seaborn

b. Matplotlib

My focus now is to see for myself what kind of relationship the data has with other data that may belong to the same or different category. 
So how many made the cut? Turns out very few lived, 61% Died. Remember this is just for training data. In reality it was even worst 68% people on board died. 

In [ ]:
df.groupby(df['Survived']).count()



In [ ]:
549/(549+342)

In [ ]:
#plot how many survive
names = 'Died', 'Survived'
size = list(dict(Counter(df['Survived'])).values())
fig = plt.figure(figsize=(18,8))
fig.patch.set_facecolor('black')
plt.rcParams['text.color'] = 'white'
my_circle=plt.Circle( (2,2), 0.9, color='black')
plt.pie(size, labels=names)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.legend(loc='lower left')
plt.title("Training Data- 'Died vs Survived' ")
plt.show()

In [ ]:
names = 'Mr', 'Mrs', 'Miss', 'Master'
size = list(dict(Counter(df['Title'])).values())
fig = plt.figure(figsize=(18,8))
fig.patch.set_facecolor('black')
plt.rcParams['text.color'] = 'white'
my_circle=plt.Circle( (3,3), 0.9, color='black')
plt.pie(size, labels=names)
p=plt.gcf()
p.gca().add_artist(my_circle)
plt.legend(loc='lower left')
plt.show()


In [ ]:
plt.rcParams['figure.figsize'] = (18, 7)
plt.style.use('seaborn-dark-palette')
sns.countplot(df['Title'], palette = 'rainbow')
plt.title('Title Variations', fontsize = 20)

In [ ]:
# Title,PClass vs Survived
f,ax=plt.subplots(1,2,figsize=(25,10))
df.groupby(['Title','Survived'])['Title'].count().plot.bar(ax=ax[0])
ax[0].set_title("Title vs Survival/Death")
df.groupby(['Pclass','Survived'])['Pclass'].count().plot.bar(ax=ax[1])
ax[1].set_title('Passenger Class vs Survival')



In [ ]:
f1,ax1=plt.subplots(1,2,figsize=(25,10))
df.groupby(['Embarked','Survived'])['Embarked'].count().plot.bar(ax=ax1[0])
ax1[0].set_title("Embarked vs Survival/Death")
df.groupby(['Sex','Survived'])['Sex'].count().plot.bar(ax=ax1[1])
ax1[1].set_title('Sex vs Survival')

In [ ]:
sns.pairplot(df, kind="scatter",hue= 'Embarked', markers=["o", "s", "D"],palette="Set2")
plt.show()

In [ ]:
sns.relplot(x="PassengerId", y="Age", hue="Title", data=df)

In [ ]:
sns.relplot(x="PassengerId", y="Age", hue="Embarked",col="Title",col_wrap = 2, palette="ch:r=-.5,l=.75", data=df)

In [ ]:


# Fitting the models
select_model = []

from sklearn.preprocessing import LabelEncoder


LE = LabelEncoder()
X['Embarked'] = LE.fit_transform(X['Embarked'])
X["Sex"] = LE.fit_transform(X["Sex"])
X['Title'] = LE.fit_transform(X['Title'])
Test['Embarked'] = LE.fit_transform(Test['Embarked'])
Test["Sex"] = LE.fit_transform(Test["Sex"])
Test['Title'] = LE.fit_transform(Test['Title'])

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.1,random_state =0)



In [ ]:
X.describe()

In [ ]:
colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)

sns.heatmap(X.astype(float).corr(),linewidths=0.1,vmax=1.0, 
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
pd.crosstab([df.Title,df.Pclass,df.Embarked],
            [df.Sex,df.Survived],margins=True).style.background_gradient(cmap='summer_r')


# 3. Modelling 

The process of training an ML model involves providing an ML algorithm (that is, the learning algorithm) with training data to learn from.
The term ML model refers to the model artifact that is created by the training process.
The training data must contain the correct answer, which is known as a target or target attribute. 
The learning algorithm finds patterns in the training data that map the input data attributes to the target 
(the answer that you want to predict), and it outputs an ML model that captures these patterns.

You can use the ML model to get predictions on new data for which you do not know the target. 
For example, let's say that you want to train an ML model to predict if an email is spam or not spam. 
You would provide ML with training data that contains emails for which you know the target (that is, a label that tells whether an email is spam or not spam). 
ML would train an ML model by using this data, resulting in a model that attempts to predict whether new email will be spam or not spam.

The model in this case is a ***Binary Classification*** problem meaning the output of this model is either a class 0 or a class 1.

## 3a. Logistic Regression

Logistic regression produces a logistic curve, which is limited to values between 0 and 1. Logistic regression is similar to a linear regression, but the curve is constructed using the natural logarithm of the “odds” of the target variable, rather than the probability. Moreover, the predictors do not have to be normally distributed or have equal variance in each group.

The logistic function, also called the sigmoid function was developed by statisticians to describe properties of population growth in ecology, rising quickly and maxing out at the carrying capacity of the environment. It’s an S-shaped curve that can take any real-valued number and map it into a value between 0 and 1, but never exactly at those limits.
                                                        
                                                        


![#Sigmoidal](https://i.imgur.com/FnooHjb.png)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix   


LR = LogisticRegression()
# LR.fit(x_train,y_train)
# y_pred = LR.predict(x_test)
# confusion_matrix(y_test,y_pred)
# print(LR.score(x_test,y_test))
LR.fit(X,Y)

#select_model.append(LR.score(x_test,y_test))


## 3b. Gaussian Naive Bayes

A Gaussian Naive Bayes algorithm is a special type of NB algorithm. It’s specifically used when the features have continuous values. It’s also assumed that all the features are following a gaussian distribution i.e, normal distribution. We use sklearn’s GaussianNB module but first lets recap to what Bayes theorem does

Let’s break the equation(below) down:
1. A and B are events.
2. P(A) and P(B) (P(B) not 0) are the probabilities of the event independent from each other.
3. P(A|B) is the probability A under the condition B.
4. Equivalent with P(B|A), it is the probability of observing event B given that event A is true.


Gaussian Naive Bayes
The Gaussian Naive Bayes is one classifier model. Beside the Gaussian Naive Bayes there are also existing the Multinomial naive Bayes and the Bernoulli naive Bayes. I picked the Gaussian Naive Bayes because it is the simplest and the most popular one.

Remember that P(C) is not calculated for our data. We assume that P(C|A) is Guassian(following normal distribution).

![Bayes Thorem](https://i.imgur.com/WGrc6ET.jpg)

In [ ]:
from sklearn.naive_bayes import GaussianNB

GBayes_clf = GaussianNB()
#GBayes_clf.fit(x_train, y_train)
GBayes_clf.fit(X, Y)
# print (GBayes_clf.score(x_test,y_test))
# select_model.append(GBayes_clf.score(x_test,y_test))
# confusion_matrix(y_pred,y_test)

## 3c. K Neighbors Classifier:

k-Nearest-Neighbors (k-NN) is a supervised machine learning model. Supervised learning is when a model learns from data that is already labeled. A supervised learning model takes in a set of input objects and output values. The model then trains on that data to learn how to map the inputs to the desired output so it can learn to make predictions on unseen data.
k-NN models work by taking a data point and looking at the ‘k’ closest labeled data points. The data point is then assigned the label of the majority of the ‘k’ closest points.
For example, if k = 5, and 3 of points are ‘green’ and 2 are ‘red’, then the data point in question would be labeled ‘green’, since ‘green’ is the majority.



![KNN](https://i.imgur.com/wZKzMPX.gif)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

k_range = range(1,31)
weights_options=['uniform','distance']
param = {'n_neighbors':k_range, 'weights':weights_options}

cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
grid_KNNsearch = GridSearchCV(KNeighborsClassifier(), param, cv=cv) 
# grid_KNNsearch.fit(x_train,y_train)
grid_KNNsearch.fit(X,Y)
grid_KNNsearch = grid_KNNsearch.best_estimator_
# y_pred = grid_KNNsearch.predict(x_test)
# print(grid_KNNsearch.score(x_test,y_test))

# select_model.append(grid_KNNsearch.score(x_test,y_test))

# confusion_matrix(y_pred,y_test)

## 3d. Support Vector Machine
A Support Vector Machine (SVM) is a supervised machine learning algorithm that can be employed for both classification and regression purposes. SVMs are more commonly used in classification problems and as such, this is what we will focus on in this post.
In this algorithm, we plot each data item as a point in n-dimensional space (where n is number of features you have) with the value of each feature being the value of a particular coordinate. Then, we perform classification by finding the optimal hyper-plane that differentiate the two classes very well.
Hyper-plane will be a point in case of 1 dimensional data, line in case of 2 dimensional data, plane in case of 3 dimensional data and so on.

![SVM](https://i.imgur.com/UNJZ7C2.png)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
Cs = [0.001, 0.01, 0.1, 1,1.5,2,2.5,3,4,5, 10] 
gammas = [0.0001,0.001, 0.01, 0.1, 1]
param_grid = {'C': Cs, 'gamma' : gammas}
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
grid_search = GridSearchCV(SVC(kernel = 'rbf', probability=True), param_grid, cv=cv) 
# grid_search.fit(x_train,y_train)
grid_search.fit(X,Y)
SVC_grid = grid_search.best_estimator_
cross_val_score(SVC_grid, X, Y, cv=5).mean()
# print(SVC_grid.score(x_test,y_test))
# select_model.append(SVC_grid.score(x_test,y_test))

## 3e. Decision Tree
A decision tree is a tree where each node represents a feature(attribute), each link(branch) represents a decision(rule) and each leaf represents an outcome(categorical or continues value).

The whole idea is to create a tree like this for the entire data and process a single outcome at every leaf(or minimize the error in every leaf).

There are couple of algorithms there to build a decision tree , we only talk about a few which are:
1. CART (Classification and Regression Trees) → uses Gini Index(Classification) as metric.
2. ID3 (Iterative Dichotomiser 3) → uses Entropy function and Information gain as metrics.

![Decidion Tree](https://i.imgur.com/yJ68HrC.png)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
    
tree_clf = DecisionTreeClassifier(max_depth = 5)
tree_clf.fit(X,Y)
score_tree = cross_val_score(tree_clf, X, Y, cv=5).mean()
print(score_tree)
select_model.append(score_tree)

In [ ]:

from sklearn.ensemble import BaggingClassifier
n_estimators = [10,30,50,70,80,150,160, 170,175,180,185];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)

params = {'n_estimators':n_estimators}

bagging_grid = GridSearchCV(BaggingClassifier(base_estimator= None,
                                      bootstrap_features=False),
                                 param_grid=params,
                                 cv=cv,
                                 n_jobs = -1)
bagging_grid.fit(X,Y)
bagging_estimator = bagging_grid.best_estimator_

bagging_estimator_score = cross_val_score(bagging_estimator, X, Y, cv=5).mean()



In [ ]:
cross_val_score(bagging_estimator, X, Y, cv=5).mean()

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
GaussianProcessClassifier = GaussianProcessClassifier()
GaussianProcessClassifier.fit(X,Y)
# y_pred = GaussianProcessClassifier.predict(x_test)
# print(accuracy_score(y_pred, y_test))
# select_model.append(accuracy_score(y_pred, y_test))

cross_val_score(GaussianProcessClassifier, X, Y, cv=5).mean()

In [ ]:

from sklearn.ensemble import ExtraTreesClassifier
ExtraTreesClassifier = ExtraTreesClassifier()
ExtraTreesClassifier.fit(X, Y)
# y_pred = ExtraTreesClassifier.predict(x_test)
# extraTree_accy = round(accuracy_score(y_pred, y_test), 3)
# select_model.append(extraTree_accy)
# print(extraTree_accy)

cross_val_score(ExtraTreesClassifier, X, Y, cv=5).mean()


In [ ]:
from xgboost.sklearn import XGBClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

params={
    'max_depth': [5],
    'subsample': [1.0],
    'colsample_bytree': [0.5],
    'n_estimators': [1000],
    'reg_alpha': [0.01, 0.02, 0.03, 0.04]
}

xgb_clf = XGBClassifier()
grid_search = GridSearchCV(xgb_clf,
                  params,
                  cv=5,
                  n_jobs=1,
                  verbose=2)


grid_search.fit(X,Y)
#print("\nGrid Search Best parameters set :")
#print(grid_search.best_params_)



In [ ]:
model = grid_search.best_estimator_

cross_val_score(model, X, Y, cv=5).mean()

In [ ]:


from sklearn.ensemble import RandomForestClassifier
n_estimators = [140,145,150,155,160];
max_depth = range(1,10);
criterions = ['gini', 'entropy'];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)


parameters = {'n_estimators':n_estimators,
              'max_depth':max_depth,
              'criterion': criterions
              
        }
grid = GridSearchCV(estimator=RandomForestClassifier(max_features='auto'),
                                 param_grid=parameters,
                                 cv=cv,
                                 n_jobs = -1)
grid.fit(X,Y) 
rf_grid = grid.best_estimator_
# print(rf_grid.score(x_test,y_test))
# select_model.append(rf_grid.score(x_test,y_test))


In [ ]:
cross_val_score(rf_grid, X, Y, cv=5).mean()

In [ ]:

from sklearn.ensemble import AdaBoostClassifier
n_estimators = [100,140,145,150,160, 170,175,180,185];
cv = StratifiedShuffleSplit(n_splits=10, test_size=.30, random_state=15)
learning_r = [0.1,1,0.01,0.5]

parameters = {'n_estimators':n_estimators,
              'learning_rate':learning_r
              
        }
adaBoost_grid = GridSearchCV(AdaBoostClassifier(base_estimator= None, ),
                                 param_grid=parameters,
                                 cv=cv,
                                 n_jobs = -1)
adaBoost_grid.fit(X,Y) 
AdaBoost_estimator = adaBoost_grid.best_estimator_
# print(AdaBoost_estimator.score(x_test,y_test))
# select_model.append(AdaBoost_estimator.score(x_test,y_test))


In [ ]:
cross_val_score(AdaBoost_estimator, X, Y, cv=5).mean()

In [ ]:
from sklearn.ensemble import VotingClassifier

voting_classifier = VotingClassifier(estimators=[
    ('lr_grid', LR),
    ('svc', SVC_grid),('AdaBoost', AdaBoost_estimator),
    ('random_forest', rf_grid),
    ('knn_classifier', grid_KNNsearch),
    ('bagging_classifier', bagging_grid),
    ('Decision Tree', tree_clf),('XGBClassifier',model),
    ('ExtraTrees_Classifier', ExtraTreesClassifier),
    ('gaussian_bayes_classifier',GBayes_clf),
    ('gaussian_process_classifier', GaussianProcessClassifier)
],voting='hard')

voting_classifier = voting_classifier.fit(X,Y)   
# y_pred = voting_classifier.predict(x_test)
# voting_accy = accuracy_score(y_pred, y_test)
# print(voting_accy)
# select_model.append(voting_accy)


In [ ]:
cross_val_score(voting_classifier, X, Y, cv=5).mean()

In [ ]:
test_prediction = voting_classifier.predict(Test[features])

In [ ]:
submission = pd.DataFrame({
        "PassengerId": Test['PassengerId'],
        "Survived": test_prediction
    })
print(submission)
submission.to_csv('submission.csv', index=False)